# Tutorial 3: Periodic Unit Cell Analysis

This notebook demonstrates analyzing periodic structures (metasurfaces, FSS, phased array unit cells).

**Learning objectives:**
- Create unit cell geometry with periodic boundaries
- Compute reflection/transmission coefficients
- Analyze oblique incidence behavior
- Extract surface impedance

In [ ]:
import sys
sys.path.insert(0, '../build/python')
sys.path.insert(0, '../python')

from vectorem.designs import UnitCellDesign
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

## 1. Create Unit Cell

Design a simple square patch unit cell for 10 GHz:

In [ ]:
# Unit cell parameters
period = 5e-3        # 5mm period (~λ/6 at 10 GHz)
h_sub = 0.5e-3       # 0.5mm substrate
eps_r = 3.5          # Low-loss dielectric

# Create unit cell with ground plane
cell = UnitCellDesign(
    period_x=period,
    period_y=period,
    substrate_height=h_sub,
    substrate_eps_r=eps_r,
    has_ground_plane=True,
)

print(f"Unit cell: {period*1000:.1f} x {period*1000:.1f} mm")
print(f"Substrate: h={h_sub*1000:.2f}mm, εr={eps_r}")
print(f"Ground plane: Yes (reflective metasurface)")

## 2. Add Patch Element

In [ ]:
# Square patch (80% fill factor)
patch_size = 0.8 * period

cell.add_patch(
    width=patch_size,
    length=patch_size,
    center_x=0,
    center_y=0,
)

print(f"Patch: {patch_size*1000:.1f} x {patch_size*1000:.1f} mm")
print(f"Fill factor: {(patch_size/period)**2 * 100:.0f}%")

## 3. Generate Mesh

In [ ]:
# Generate mesh with periodic boundaries
cell.generate_mesh(density=20, design_freq=10e9)

print(f"Mesh: {len(cell.mesh.elements)} elements")

## 4. Normal Incidence Reflection

In [ ]:
# Normal incidence at 10 GHz
R, T = cell.reflection_transmission(10e9, theta=0, phi=0, pol='TE')

print(f"Normal incidence at 10 GHz:")
print(f"  R = {R:.4f}")
print(f"  |R| = {abs(R):.4f} ({20*np.log10(abs(R)+1e-10):.1f} dB)")
print(f"  Phase(R) = {np.angle(R)*180/np.pi:.1f}°")
print(f"\n  T = {T:.4f} (should be ~0 for grounded structure)")

## 5. Frequency Sweep

In [ ]:
# Frequency sweep
freqs, R_arr, T_arr = cell.frequency_sweep_reflection(
    f_start=5e9,
    f_stop=15e9,
    n_points=41,
    theta=0,
    phi=0,
    pol='TE',
    verbose=True
)

# Find resonance (minimum |R| or phase crossing)
R_mag = np.abs(R_arr)
idx_min = np.argmin(R_mag)
f_res = freqs[idx_min]

print(f"\nResonance: {f_res/1e9:.2f} GHz")
print(f"  |R|_min = {R_mag[idx_min]:.4f}")

In [ ]:
# Plot reflection vs frequency
fig, axes = plt.subplots(2, 1, figsize=(10, 8))

# Magnitude
axes[0].plot(freqs/1e9, 20*np.log10(R_mag+1e-10), 'b-', linewidth=2)
axes[0].axvline(x=f_res/1e9, color='r', linestyle='--', label=f'Resonance: {f_res/1e9:.1f} GHz')
axes[0].set_xlabel('Frequency (GHz)')
axes[0].set_ylabel('|R| (dB)')
axes[0].set_title('Unit Cell Reflection Magnitude')
axes[0].legend()
axes[0].grid(True)

# Phase
R_phase = np.angle(R_arr) * 180 / np.pi
axes[1].plot(freqs/1e9, R_phase, 'b-', linewidth=2)
axes[1].axhline(y=0, color='r', linestyle='--', alpha=0.5)
axes[1].axhline(y=-180, color='g', linestyle=':', alpha=0.5)
axes[1].set_xlabel('Frequency (GHz)')
axes[1].set_ylabel('Phase(R) (deg)')
axes[1].set_title('Reflection Phase')
axes[1].grid(True)

plt.tight_layout()
plt.show()

## 6. Surface Impedance

In [ ]:
# Surface impedance at several frequencies
test_freqs = [8e9, 10e9, 12e9]
Z0 = 376.73  # Free-space impedance

print("Surface Impedance:")
print(f"{'Freq (GHz)':<12} {'Zs (Ω)':<25} {'Zs/Z0':<20}")
print("-" * 57)

for f in test_freqs:
    Zs = cell.surface_impedance(f)
    Zs_norm = Zs / Z0
    print(f"{f/1e9:<12.1f} {Zs.real:>8.1f}+j{Zs.imag:<12.1f} {Zs_norm.real:>6.3f}+j{Zs_norm.imag:<10.3f}")

## 7. Oblique Incidence

In [ ]:
# Scan angle sweep at resonant frequency
angles = np.arange(0, 61, 5)
R_te = []
R_tm = []

print(f"Scanning at {f_res/1e9:.1f} GHz:")
for theta in angles:
    r_te, _ = cell.reflection_transmission(f_res, theta=theta, phi=0, pol='TE')
    r_tm, _ = cell.reflection_transmission(f_res, theta=theta, phi=0, pol='TM')
    R_te.append(r_te)
    R_tm.append(r_tm)
    print(f"  θ={theta:2d}°: |R_TE|={abs(r_te):.3f}, |R_TM|={abs(r_tm):.3f}")

R_te = np.array(R_te)
R_tm = np.array(R_tm)

In [ ]:
# Plot angular dependence
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Magnitude
axes[0].plot(angles, np.abs(R_te), 'b-o', label='TE')
axes[0].plot(angles, np.abs(R_tm), 'r--s', label='TM')
axes[0].set_xlabel('Incidence Angle (deg)')
axes[0].set_ylabel('|R|')
axes[0].set_title(f'Reflection Magnitude at {f_res/1e9:.1f} GHz')
axes[0].legend()
axes[0].grid(True)

# Phase
axes[1].plot(angles, np.angle(R_te)*180/np.pi, 'b-o', label='TE')
axes[1].plot(angles, np.angle(R_tm)*180/np.pi, 'r--s', label='TM')
axes[1].set_xlabel('Incidence Angle (deg)')
axes[1].set_ylabel('Phase(R) (deg)')
axes[1].set_title('Reflection Phase')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

## 8. Reflection Phase Gradient

For metasurface design, we often want to control reflection phase vs. frequency:

In [ ]:
# Phase vs frequency (detailed)
freqs_fine, R_fine, _ = cell.frequency_sweep_reflection(
    f_start=8e9, f_stop=12e9, n_points=81
)

# Unwrap phase for continuous curve
phase_fine = np.unwrap(np.angle(R_fine)) * 180 / np.pi

plt.figure(figsize=(10, 5))
plt.plot(freqs_fine/1e9, phase_fine, 'b-', linewidth=2)
plt.axhline(y=0, color='r', linestyle='--', alpha=0.5, label='0° (AMC)')
plt.axhline(y=-180, color='g', linestyle=':', alpha=0.5, label='-180° (PEC)')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Reflection Phase (deg)')
plt.title('Reflection Phase vs Frequency')
plt.legend()
plt.grid(True)
plt.show()

# Find AMC frequency (phase = 0°)
idx_amc = np.argmin(np.abs(phase_fine - 0))
f_amc = freqs_fine[idx_amc]
print(f"AMC frequency (phase=0°): {f_amc/1e9:.2f} GHz")

## 9. Export Results

In [ ]:
# Export to Touchstone
cell.export_touchstone("unit_cell_reflection", freqs, R_arr)
print("Saved: unit_cell_reflection.s1p")

## 10. Varying Patch Size

Study how patch size affects resonance:

In [ ]:
# Parametric study: patch size
fill_factors = [0.5, 0.6, 0.7, 0.8, 0.9]
colors = plt.cm.viridis(np.linspace(0, 1, len(fill_factors)))

plt.figure(figsize=(10, 6))

for ff, color in zip(fill_factors, colors):
    # Create unit cell with different patch size
    c = UnitCellDesign(
        period_x=5e-3, period_y=5e-3,
        substrate_height=0.5e-3,
        substrate_eps_r=3.5,
    )
    patch_size = ff * 5e-3
    c.add_patch(width=patch_size, length=patch_size)
    c.generate_mesh(density=15)
    
    # Quick frequency sweep
    f, R, _ = c.frequency_sweep_reflection(6e9, 14e9, n_points=31)
    
    plt.plot(f/1e9, np.angle(R)*180/np.pi, '-', color=color, 
             label=f'FF={ff*100:.0f}%', linewidth=2)

plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.xlabel('Frequency (GHz)')
plt.ylabel('Reflection Phase (deg)')
plt.title('Effect of Fill Factor on Reflection Phase')
plt.legend()
plt.grid(True)
plt.show()

## Summary

In this tutorial we:

1. Created a periodic unit cell with Floquet boundaries
2. Computed reflection coefficient vs. frequency
3. Found resonance and AMC frequency
4. Analyzed oblique incidence (TE and TM)
5. Extracted surface impedance
6. Studied effect of patch size on response

**Key concepts:**
- AMC (Artificial Magnetic Conductor): Phase(R) = 0°
- PEC reflection: Phase(R) = -180°
- Larger patch → lower resonant frequency

**Next steps:**
- [Tutorial 4: Phased Array](04_phased_array.ipynb) - Active impedance analysis